### Imports

In [ ]:
import sys
import os

project_path = os.path.abspath(os.path.join('../'))
if project_path not in sys.path:
    sys.path.append(project_path)

from utils.load_data import load_fer2013, parse_fer2013, \
                            load_CKPlus, fer_csv_to_png
from utils.generators import init_resnet_generator, init_cnn_generator
from utils.plots import plot_model_history
from models.build import build_resnet_model, build_cnn_baseline_model

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from keras.models import load_model

import numpy as np

### Loading datasets

In [ ]:
train_data, val_data, test_data = load_fer2013(file_path='../data/fer2013/fer2013.csv')
X_train, Y_train = parse_fer2013(train_data)
X_val, Y_val = parse_fer2013(val_data)
X_test, Y_test = parse_fer2013(test_data)

### Model training parameters

In [ ]:
batch_size = 128
num_epochs = 150
patience = 20
saved_models_path = '../models/saved_models/'
dataset_name = 'fer2013'
model_name = '_simple_CNN_1'
history_name = '_hist' + model_name

# ../models/saved_models/fer2013_simple_CNN_1
model_path = saved_models_path + dataset_name + model_name

# ../models/saved_models/fer2013_hist_simple_CNN_1
history_path = saved_models_path + dataset_name + history_name

### Data generators

In [ ]:
train_gen = init_cnn_generator(with_aug=True)
val_gen = init_cnn_generator(with_aug=False)

### Callbacks

In [ ]:
cpt_path = model_path + '.hdf5'
early_stop = EarlyStopping('val_loss', patience=patience)
model_checkpoint = ModelCheckpoint(cpt_path, save_best_only=True)
callbacks = [model_checkpoint, early_stop]

### Build and compile model

In [ ]:
model = build_cnn_baseline_model()
opt = Adam(0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

### Training

In [ ]:
history = model.fit(train_gen.flow(X_train, Y_train),
                    validation_data=val_gen.flow(X_val, Y_val),
                    epochs=3,
                    steps_per_epoch=len(X_train)//batch_size,
                    validation_steps=len(X_val)//batch_size,
                    callbacks=callbacks)

# Saving final model and history
val_acc = history.history['val_accuracy'][-1]
cnt_epoch = len(history.history['val_accuracy'])
np.save(f'{history_path}-e{cnt_epoch:02d}-a{val_acc:.2f}.npy', history.history)
model.save(f'{model_path}-e{cnt_epoch:02d}-a{val_acc:.2f}.hdf5')

### Loading (if needed)

In [ ]:
# Uncomment to load
# history = np.load(history_path + '-e03-a0.41.npy', allow_pickle=True)
# history = history.item()
# model = load_model(model_path + '-e88-a0.65.hdf5')

### Evaluation

In [ ]:
X_test = X_test / 255.0
loss, acc = model.evaluate(X_test, Y_test)
print("Model accuracy: {:5.2f}%".format(100 * acc))

### Ploting curves

In [ ]:
plot_model_history(history)